In [1]:
import pandas as pd
from ntscraper import Nitter

In [2]:
Scraper = Nitter()

Testing instances: 100%|██████████| 77/77 [01:44<00:00,  1.36s/it]


In [3]:
#tweets =Scraper.get_tweets("realDonaldTrump",mode ='user',number = 100)
def scraping(input):
    tweets =Scraper.get_tweets(input,mode ='user',number = 500)
    return tweets

tweets = scraping("BarackObama")

29-Apr-24 02:42:58 - No instance specified, using random instance https://nitter.esmailelbob.xyz
29-Apr-24 02:43:04 - Current stats for BarackObama: 20 tweets, 0 threads...
29-Apr-24 02:43:09 - Current stats for BarackObama: 40 tweets, 0 threads...
29-Apr-24 02:43:13 - Current stats for BarackObama: 60 tweets, 0 threads...
29-Apr-24 02:43:18 - Current stats for BarackObama: 80 tweets, 0 threads...
29-Apr-24 02:43:22 - Current stats for BarackObama: 100 tweets, 0 threads...
29-Apr-24 02:43:27 - Current stats for BarackObama: 120 tweets, 0 threads...
29-Apr-24 02:43:31 - Current stats for BarackObama: 140 tweets, 0 threads...
29-Apr-24 02:43:36 - Current stats for BarackObama: 160 tweets, 0 threads...
29-Apr-24 02:43:40 - Current stats for BarackObama: 180 tweets, 0 threads...
29-Apr-24 02:43:45 - Current stats for BarackObama: 200 tweets, 0 threads...
29-Apr-24 02:43:49 - Current stats for BarackObama: 220 tweets, 0 threads...
29-Apr-24 02:43:53 - Current stats for BarackObama: 240 twee

In [4]:
final_tweets = []
for tweet in tweets['tweets']:
    #data = [tweet['link'],tweet['text'],tweet['date'],tweet['stats']['likes'],tweet['stats']['comments']]
    data = [tweet['text'],tweet['date']]
    final_tweets.append(data)

In [5]:
data =pd.DataFrame(final_tweets, columns= ['text','date'])
#columns= ['link','text','date','no_likes','comment'])

In [6]:
data['date'] = pd.to_datetime(data['date'], format='%b %d, %Y · %I:%M %p %Z')
data['date'] = data['date'].dt.to_period('M')


/var/folders/wh/d15mczds4wxfm8qht9nw7p_w0000gn/T/ipykernel_2395/4118179080.py:2: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  data['date'] = data['date'].dt.to_period('M')


In [7]:
data.tail()

,text,date
495,I am shocked and saddened by the assassination...,2022-07
496,"Right now, thousands of Ukrainian families are...",2022-07
497,Michelle and I mourn the passing of Clifford A...,2022-07
498,"Happy birthday, Malia! No matter how sophistic...",2022-07
499,"Happy Fourth of July! As we celebrate today,...",2022-07


In [8]:
import dill
def clean(text):
    import re
    import nltk
    from nltk.util import pr
    stemmer = nltk.SnowballStemmer("english")
    from nltk.corpus import stopwords
    import string
    stopword = set(stopwords.words("english"))
    text = str(text).lower()
    text = re.sub('[.?]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w\d\w', '', text)
    text = [word for word in text.split(' ') if word not in stopword]
    text = " ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text = " ".join(text)
    return text

with open('../backend/textback/models/clean_function.pkl', 'wb') as f:
     dill.dump(clean, f)

In [9]:
import pickle
model = pickle.load(open("../backend/textback/models/model.pkl","rb"))
cv = pickle.load(open("../backend/textback/models/vectorizer.pkl","rb"))

def predict(input):
    input = cv.transform([input]).toarray()
    predict = model.predict(input)[0]
    return predict

In [10]:
tweets_cleaned = []
Negative = 0
Nuetral = 0
for i in range(len(final_tweets)):
    clean_tweets = clean(final_tweets[i])
    if clean_tweets != "":
        tweets_cleaned.append(clean_tweets)


for i in range(len(tweets_cleaned)):
    prediction = predict(tweets_cleaned[i])
    if prediction == "No hate and offensive speech":
        Nuetral += 1
    elif prediction == "Offensive language detected":
        Negative += 1
    else :
       Negative += 1

#keep the score for each text in the dataframe file
data['score'] = data['text'].apply(predict)

#change score to numeric form 0,1,2
data['score'] = data['score'].replace({'No hate and offensive speech':0,'Offensive language detected':1,'Hate speech Detected':2})
print("Score about Negative and Nuetral in This user is\n Negative = " , Negative , "\n Nuetral = " , Nuetral)


Score about Negative and Nuetral in This user is
 Negative =  123 
 Nuetral =  377


In [11]:
#data to csv file
data['Month'] = data['date'].dt.month
data.to_csv('result.csv',index=False)